In [1]:
import os.path as op
import os

### ensure workbench is installed on pc
os.environ['PATH'] += ":/Applications/workbench/bin_linux64"

In [2]:
from neuromaps.transforms import fsaverage_to_fslr
from nibabel.gifti import GiftiDataArray
from nilearn import datasets
from nilearn.plotting import plot_roi
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

In [20]:
### modify file paths as needed
regions_dir = "./data/white-matter-atlas_thresholds/1062_cortexmap_binarize_smooth-surf-1_threshold-0.15_dilate-0/cortexmap/func/"
surf_template = "./data/S1200.L.midthickness_MSMAll.32k_fs_LR.surf.gii"
vol_template = "./data/S1200_AverageT1w_restore.nii.gz"

In [21]:
file_names = []
for f in os.listdir(regions_dir):
    if f.endswith(".gii"):
        file_names.append(f)
file_names

# extract tract name from file name
for i in file_names:
    if ("rh.left" in i) or ("lh.right" in i) or ('spino' in i) or ('Cerebellar' in i):
        continue # exclude if file name "rh.left..." or "lh.right..." or spinal/cerebellar tracts
    else:
        #tracts_names.append(i)
        tract_fn = i
        map_lh = op.join(regions_dir, f"{tract_fn}")#.func.gii")
        label_fn = op.join(regions_dir, f"{tract_fn}.dlabel.gii")
        vol_fn = op.join(regions_dir, f"{tract_fn}.nii.gz")

        # Using linear resampling for continuous maps and nearest neighbor for labels
        map_fslr_lh = fsaverage_to_fslr(map_lh, target_density='32k', hemi="L", method='linear')[0]
        map_fslr_arr_lh = map_fslr_lh.agg_data()
        map_fslr_arr_lh[map_fslr_arr_lh > 0] = 1
        map_fslr_arr_lh = map_fslr_arr_lh.astype(np.int32)
        map_fslr_lh.remove_gifti_data_array(0)
        map_fslr_lh.add_gifti_data_array(GiftiDataArray(map_fslr_arr_lh))
        nib.save(map_fslr_lh, label_fn)

        cmd = f"wb_command -label-to-volume-mapping \
            {label_fn} \
            {surf_template} \
            {vol_template} \
            {vol_fn} \
            -nearest-vertex 1"
        os.system(cmd)
        cmd



In [19]:
### use if wanting to view brain plots
# template = datasets.load_mni152_template(resolution=1)
# # template = nib.load(vol_template)
# plot_roi(
#     vol_fn,
#     bg_img=template,
#     black_bg=False,
#     draw_cross=False,
#     alpha=0.8,
#     colorbar=True,
#     display_mode="mosaic",
# )
# plt.show()
# print(vol_fn)